# WZK - Zadanie 4

Autor: Dariusz Max Adamski - nr indeksu 136674

---

In [5]:
import random
from random import getrandbits
from math import gcd
from miller_rabin import miller_rabin
from Crypto.Cipher import AES
from tqdm.notebook import tqdm

pad = lambda x: x + b'\0' * (AES.block_size - len(x) % AES.block_size)
strip = lambda x: x.rstrip(b'\0')
hexdump = lambda x: ' '.join(f'{b:02X}' for b in x)

In [11]:
def randwhile(bits, until):
    while True:
        x = getrandbits(bits)
        if until(x): return x
        
def primroots(p):
    coprime = {x for x in range(1, p) if gcd(x, p) == 1}
    return [x for x in tqdm(range(1, p)) if coprime == {pow(x, y, p) for y in range(1, p)}]

n = randwhile(10, miller_rabin)
g = random.choice(primroots(n))
print(f"n = {n}, g = {g}")


n = 43, g = 5


In [12]:
x = getrandbits(80)
X = pow(g, x, n)
print(f"A: x = {x}, X = {X}")

y = getrandbits(80)
Y = pow(g, y, n)
print(f"B: y = {y}, Y = {Y}")

print('A -X--> B')
print('B -Y--> A')

Ak = pow(Y, x, n)
print(f"A: full key = {Ak}")

Bk = pow(X, y, n)
print(f"B: full key = {Bk}")

assert Ak == Bk
k = Ak = Bk

A: x = 175565266149031796200044, X = 1
B: y = 475898292913556375537608, Y = 24
A -X--> B
B -Y--> A
A: full key = 1
B: full key = 1


In [13]:
def encrypt(key, m):
    cipher = AES.new(key.to_bytes(16, byteorder='big'), AES.MODE_ECB)
    return cipher.encrypt(pad(bytes(m, 'utf-8')))

msg = encrypt(k, 'Hello, world!')
print('A: encrypted =', hexdump(msg))

A: encrypted = A7 F8 DC FD E7 1A 19 3B 0D 9D AD 9E B3 9A 50 02


In [14]:
def decrypt(key, m):
    cipher = AES.new(key.to_bytes(16, byteorder='big'), AES.MODE_ECB)
    return strip(cipher.decrypt(m)).decode('utf-8')

res = decrypt(k, msg)
print('B: decrypted =', res)

B: decrypted = Hello, world!
